In [ ]:
# Install lightweight dependencies only (avoiding torch for faster setup)
pip install psycopg2-binary sqlalchemy pandas nltk scikit-learn tqdm

In [ ]:
# Optional: Install heavy ML dependencies only if needed
# Uncomment the line below only if you need advanced NLP features
# !pip install torch sentence-transformers keybert --index-url https://download.pytorch.org/whl/cpu

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import psycopg2
from psycopg2.extras import RealDictCursor
import json

# Try to import torch, fallback to CPU-only processing if not available
try:
    import torch
    torch_available = True
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        device = 'cuda'
    else:
        print("Using CPU with PyTorch")
        device = 'cpu'
except ImportError:
    torch_available = False
    device = 'cpu'
    print("PyTorch not available - using basic text processing")

# Database connection parameters
DB_CONFIG = {
    'host': 'ep-lingering-term-ab7pbfql-pooler.eu-west-2.aws.neon.tech',
    'database': 'neondb',
    'user': 'neondb_owner',
    'password': 'npg_ExFXHY8yiNT0',
    'port': 5432,
    'sslmode': 'require'
}

# Test database connection and fetch data
print("Connecting to PostgreSQL database...")
try:
    # Connect to database
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor(cursor_factory=RealDictCursor)
    
    # Fetch data
    print("Fetching data from silver.silver_translated...")
    cur.execute("SELECT * FROM silver.silver_translated")
    rows = cur.fetchall()
    
    # Convert to DataFrame
    df = pd.DataFrame([dict(row) for row in rows])
    print(f"Dataset loaded from database: {df.shape}")
    print("Database connection successful!")
    
    # Close initial connection
    cur.close()
    conn.close()
    
except Exception as e:
    print(f"Database connection failed: {e}")
    # Create empty DataFrame as fallback
    df = pd.DataFrame()

CUDA available: True
GPU: NVIDIA GeForce GTX 1650
Connecting to PostgreSQL database...
Fetching data from silver.silver_translated...
Fetching data from silver.silver_translated...
Dataset loaded from database: (36, 16)
Database connection successful!
Dataset loaded from database: (36, 16)
Database connection successful!


In [5]:
# Data preprocessing - fill NaN values
if not df.empty:
    # Use the correct column names from the database
    df['positive_review_translated'] = df['positive_review_translated'].fillna('')
    df['negative_review_translated'] = df['negative_review_translated'].fillna('')
    print(f"Data preprocessing completed. Dataset shape: {df.shape}")
    print(f"Available columns: {list(df.columns)}")
else:
    print("No data loaded - skipping preprocessing")

Data preprocessing completed. Dataset shape: (36, 16)
Available columns: ['id', 'city', 'hotel_name', 'reviewer_name', 'reviewer_nationality', 'duration', 'check_in_date', 'travel_type', 'room_type', 'review_date', 'positive_review', 'negative_review', 'ingestion_timestamp', 'sentiment_classification', 'negative_review_translated', 'positive_review_translated']


In [6]:
# Import advanced NLP libraries only if available
try:
    from keybert import KeyBERT
    from sentence_transformers import SentenceTransformer
    keybert_available = True
    print("Advanced NLP libraries loaded successfully")
except ImportError:
    keybert_available = False
    print("Advanced NLP libraries not available - using basic keyword extraction")

import time
from sklearn.feature_extraction.text import TfidfVectorizer
import re

if keybert_available and torch_available:
    # Advanced keyword extraction with KeyBERT
    print("Loading KeyBERT models...")
    start_time = time.time()
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    kw_model = KeyBERT(model=embedding_model)
    print(f"Models loaded in {time.time() - start_time:.2f} seconds")
    
    def extract_semantic_phrases_batch(texts, batch_size=100):
        """Extract keywords using KeyBERT"""
        all_results = []
        valid_indices = []
        valid_texts = []
        
        for i, text in enumerate(texts):
            if isinstance(text, str) and len(text.strip()) > 10:
                valid_indices.append(i)
                valid_texts.append(text)

        print(f"Processing {len(valid_texts)} valid texts out of {len(texts)} total")

        valid_results = []
        for i in tqdm(range(0, len(valid_texts), batch_size), desc="Processing batches"):
            batch = valid_texts[i:i+batch_size]
            batch_results = []

            for text in batch:
                try:
                    keywords = kw_model.extract_keywords(
                        text,
                        keyphrase_ngram_range=(1, 2),
                        stop_words='english',
                        use_maxsum=False,
                        nr_candidates=10,
                        top_n=3
                    )
                    batch_results.append([kw[0] for kw in keywords])
                except Exception as e:
                    batch_results.append([])

            valid_results.extend(batch_results)

        result_map = dict(zip(valid_indices, valid_results))
        all_results = [result_map.get(i, []) for i in range(len(texts))]
        return all_results

else:
    # Fallback to TF-IDF based keyword extraction
    print("Using TF-IDF based keyword extraction...")
    
    def extract_semantic_phrases_batch(texts, batch_size=100):
        """Extract keywords using TF-IDF as fallback"""
        all_results = []
        valid_texts = []
        valid_indices = []
        
        for i, text in enumerate(texts):
            if isinstance(text, str) and len(text.strip()) > 10:
                valid_indices.append(i)
                valid_texts.append(text)
        
        if not valid_texts:
            return [[] for _ in texts]
        
        # Use TF-IDF to extract important terms
        vectorizer = TfidfVectorizer(
            max_features=1000,
            ngram_range=(1, 2),
            stop_words='english',
            min_df=1,
            max_df=0.8
        )
        
        try:
            tfidf_matrix = vectorizer.fit_transform(valid_texts)
            feature_names = vectorizer.get_feature_names_out()
            
            valid_results = []
            for i in range(len(valid_texts)):
                # Get top 3 terms for each document
                doc_scores = tfidf_matrix[i].toarray()[0]
                top_indices = doc_scores.argsort()[-3:][::-1]
                top_terms = [feature_names[idx] for idx in top_indices if doc_scores[idx] > 0]
                valid_results.append(top_terms)
        except:
            valid_results = [[] for _ in valid_texts]
        
        result_map = dict(zip(valid_indices, valid_results))
        all_results = [result_map.get(i, []) for i in range(len(texts))]
        return all_results

# Process reviews if data is available
if not df.empty:
    print("Processing positive reviews...")
    df['Semantic Phrases Pos'] = extract_semantic_phrases_batch(df['positive_review_translated'].tolist())

    print("Processing negative reviews...")
    df['Semantic Phrases Neg'] = extract_semantic_phrases_batch(df['negative_review_translated'].tolist())

    if torch_available and device == 'cuda':
        torch.cuda.empty_cache()
        print("GPU cache cleared")
else:
    print("No data to process - skipping semantic analysis")

/home/kariem/airflow/airflow_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Advanced NLP libraries loaded successfully
Loading KeyBERT models...
Models loaded in 94.56 seconds
Processing positive reviews...
Processing 29 valid texts out of 36 total
Models loaded in 94.56 seconds
Processing positive reviews...
Processing 29 valid texts out of 36 total


Processing batches: 100%|██████████| 1/1 [00:03<00:00,  3.80s/it]



Processing negative reviews...
Processing 34 valid texts out of 36 total


Processing batches: 100%|██████████| 1/1 [00:03<00:00,  3.06s/it]

GPU cache cleared


In [7]:
import re
import nltk
from collections import Counter

# Download required NLTK data
try:
    nltk.download('stopwords', quiet=True)
    nltk.download('names', quiet=True)
    from nltk.corpus import stopwords, names
    stop_words = set(stopwords.words('english'))
    name_list = set(names.words())
except:
    print("NLTK data not available, using basic filtering")
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}
    name_list = set()

# Additional hotel-specific stop words to filter
hotel_stopwords = {
    'hotel', 'room', 'stay', 'night', 'day', 'time', 'place', 'location', 'area', 'staff', 'service',
    'people', 'guest', 'customer', 'visitor', 'person', 'man', 'woman', 'guy', 'lady', 'someone',
    'anyone', 'everyone', 'everything', 'something', 'anything', 'nothing'
}

# Comprehensive tourism services and interests vocabulary
tourism_vocabulary = {
    # Accommodation & Room Features
    'accommodation', 'suite', 'apartment', 'villa', 'cottage', 'cabin', 'chalet', 'hostel', 'guesthouse', 'resort',
    'bedroom', 'bathroom', 'kitchen', 'balcony', 'terrace', 'patio', 'garden', 'pool', 'jacuzzi', 'spa',
    'wifi', 'internet', 'television', 'minibar', 'refrigerator', 'airconditioning', 'heating', 'fireplace',
    'bed', 'pillow', 'mattress', 'linen', 'towel', 'amenities', 'toiletries', 'hairdryer', 'safe', 'wardrobe',

    # Hotel Services & Facilities
    'reception', 'concierge', 'housekeeping', 'maintenance', 'security', 'valet', 'bellhop', 'porter',
    'checkin', 'checkout', 'reservation', 'booking', 'availability', 'upgrade', 'complimentary', 'inclusive',
    'restaurant', 'dining', 'breakfast', 'lunch', 'dinner', 'buffet', 'menu', 'cuisine', 'bar', 'lounge',
    'gym', 'fitness', 'sauna', 'massage', 'wellness', 'relaxation', 'treatment', 'therapy',
    'conference', 'meeting', 'business', 'events', 'wedding', 'banquet', 'catering',
    'parking', 'garage', 'transportation', 'shuttle', 'airport', 'transfer', 'taxi', 'rental',

    # Tourism Activities & Attractions
    'sightseeing', 'tour', 'excursion', 'adventure', 'exploration', 'hiking', 'walking', 'cycling', 'biking',
    'museum', 'gallery', 'exhibition', 'theater', 'cinema', 'entertainment', 'nightlife', 'shopping',
    'beach', 'ocean', 'sea', 'lake', 'river', 'mountain', 'forest', 'park', 'nature', 'landscape',
    'historic', 'cultural', 'heritage', 'architecture', 'monument', 'castle', 'church', 'temple',
    'festival', 'event', 'celebration', 'carnival', 'market', 'fair', 'concert', 'performance',

    # Experience & Service Quality
    'comfortable', 'luxury', 'elegant', 'modern', 'traditional', 'authentic', 'unique', 'spectacular',
    'clean', 'spacious', 'cozy', 'quiet', 'peaceful', 'relaxing', 'convenient', 'accessible',
    'friendly', 'helpful', 'professional', 'courteous', 'attentive', 'responsive', 'efficient',
    'delicious', 'tasty', 'fresh', 'quality', 'variety', 'selection', 'choice', 'option',
    'expensive', 'affordable', 'reasonable', 'value', 'price', 'cost', 'budget', 'cheap',
    'recommend', 'satisfaction', 'experience', 'memorable', 'enjoyable', 'pleasant', 'disappointing',

    # Location & Geography
    'downtown', 'center', 'district', 'neighborhood', 'vicinity', 'nearby', 'walking', 'distance',
    'view', 'scenery', 'panoramic', 'overlooking', 'facing', 'waterfront', 'beachfront', 'hillside',
    'accessibility', 'transportation', 'connection', 'proximity', 'convenience', 'central',

    # Negative Aspects
    'problem', 'issue', 'complaint', 'dissatisfied', 'unhappy', 'poor', 'worst', 'terrible',
    'dirty', 'noisy', 'crowded', 'outdated', 'broken', 'damaged', 'faulty',
    'rude', 'unfriendly', 'unprofessional', 'slow', 'delayed', 'cancelled', 'overbooked',
    'overpriced', 'overrated', 'disappointing', 'unacceptable', 'uncomfortable'
}

def extract_and_clean_tokens(phrase_lists):
    """Extract individual tokens from phrase lists and remove redundant/non-meaningful ones"""
    all_tokens = []

    for phrase_list in phrase_lists:
        if isinstance(phrase_list, list):
            for phrase in phrase_list:
                if isinstance(phrase, str):
                    # Split phrase into individual words
                    tokens = re.findall(r'\b[a-z]+\b', phrase.lower())
                    all_tokens.extend(tokens)

    # Remove duplicates and filter tokens
    unique_tokens = set(all_tokens)

    # Filter out unwanted tokens
    filtered_tokens = []
    for token in unique_tokens:
        if (len(token) >= 3 and
            token not in stop_words and
            token not in hotel_stopwords and
            token.lower() not in name_list and
            not token.isdigit() and
            token.isalpha()):
            filtered_tokens.append(token)

    return sorted(filtered_tokens)

# Extract and clean tokens
print("Extracting and cleaning tokens...")
positive_tokens = extract_and_clean_tokens(df['Semantic Phrases Pos'])
negative_tokens = extract_and_clean_tokens(df['Semantic Phrases Neg'])
print(f"Positive tokens extracted: {len(positive_tokens)}")
print(f"Negative tokens extracted: {len(negative_tokens)}")

Extracting and cleaning tokens...
Positive tokens extracted: 65
Negative tokens extracted: 59


In [8]:
# Filter for tourism-relevant tokens and apply to dataframe
def filter_tourism_tokens(token_list, tourism_vocab):
    """Filter tokens to only include tourism-related terms"""
    return sorted([token for token in token_list if token.lower() in tourism_vocab])

def extract_tourism_tokens_per_row(phrase_list, valid_tourism_tokens):
    """Extract only tourism-relevant tokens for each row"""
    if not isinstance(phrase_list, list):
        return []

    row_tokens = set()
    for phrase in phrase_list:
        if isinstance(phrase, str):
            tokens = re.findall(r'\b[a-z]+\b', phrase.lower())
            for token in tokens:
                if token in [t.lower() for t in valid_tourism_tokens]:
                    row_tokens.add(token)

    return sorted(list(row_tokens))

# Filter for tourism-relevant tokens
tourism_positive_tokens = filter_tourism_tokens(positive_tokens, tourism_vocabulary)
tourism_negative_tokens = filter_tourism_tokens(negative_tokens, tourism_vocabulary)

print(f"Tourism-relevant positive tokens: {len(tourism_positive_tokens)}")
print(f"Tourism-relevant negative tokens: {len(tourism_negative_tokens)}")

# Create tourism-filtered token columns
df['Tourism_Tokens_Pos'] = df['Semantic Phrases Pos'].apply(
    lambda x: extract_tourism_tokens_per_row(x, tourism_positive_tokens)
)
df['Tourism_Tokens_Neg'] = df['Semantic Phrases Neg'].apply(
    lambda x: extract_tourism_tokens_per_row(x, tourism_negative_tokens)
)

# Filter out reviews with no tourism tokens
df = df[~((df['Tourism_Tokens_Pos'].apply(len) == 0) &
          (df['Tourism_Tokens_Neg'].apply(len) == 0))].copy()

print(f"Final dataset with tourism tokens: {len(df)} reviews")

Tourism-relevant positive tokens: 14
Tourism-relevant negative tokens: 15
Final dataset with tourism tokens: 23 reviews


In [9]:
# Prepare data for database ingestion with target schema
if not df.empty:
    final_df = pd.DataFrame()

    # Map source columns to target schema using correct column names
    final_df['city'] = df['city']
    final_df['hotel_name'] = df['hotel_name']
    final_df['reviewer_name'] = df['reviewer_name']
    final_df['reviewer_nationality'] = df['reviewer_nationality']
    final_df['duration'] = df['duration']
    final_df['check_in_date'] = df['check_in_date']
    final_df['review_date'] = df['review_date']
    final_df['travel_type'] = df['travel_type']
    final_df['room_type'] = df['room_type']
    final_df['positive_review'] = df['positive_review_translated']
    final_df['negative_review'] = df['negative_review_translated']
    final_df['sentiment_classification'] = df['sentiment_classification']

    # Convert tourism tokens to PostgreSQL TEXT[] format
    def convert_to_postgres_array(token_list):
        """Convert Python list to PostgreSQL TEXT[] format"""
        if not isinstance(token_list, list) or len(token_list) == 0:
            return None
        escaped_tokens = []
        for token in token_list:
            escaped_token = str(token).replace("'", "''")
            escaped_tokens.append(f"'{escaped_token}'")
        return '{' + ','.join(escaped_tokens) + '}'

    final_df['positive_tokens'] = df['Tourism_Tokens_Pos'].apply(convert_to_postgres_array)
    final_df['negative_tokens'] = df['Tourism_Tokens_Neg'].apply(convert_to_postgres_array)

    print(f"Final dataset prepared: {final_df.shape}")
else:
    print("No data to prepare for ingestion")
    final_df = pd.DataFrame()

Final dataset prepared: (23, 14)


In [10]:
# Create target table and ingest data using psycopg2 (APPEND MODE)
print("=== DATABASE INGESTION (APPEND MODE) ===")

if not final_df.empty:
    try:
        # Connect to database
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        
        # Create gold schema if it doesn't exist
        cur.execute("CREATE SCHEMA IF NOT EXISTS gold")
        
        # Create table only if it doesn't exist (no DROP)
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS gold.final_reviews_test (
            id SERIAL PRIMARY KEY,
            city TEXT,
            hotel_name TEXT,
            reviewer_name TEXT,
            reviewer_nationality TEXT,
            duration TEXT,
            check_in_date TEXT,
            review_date TEXT,
            travel_type TEXT,
            room_type TEXT,
            positive_review TEXT,
            negative_review TEXT,
            sentiment_classification INTEGER,
            positive_tokens TEXT[],
            negative_tokens TEXT[],
            inserted_at TIMESTAMP DEFAULT now()
        )
        """

        cur.execute(create_table_sql)
        conn.commit()
        print("✅ Target table ready (created if not exists)")

        # Check existing record count
        cur.execute("SELECT COUNT(*) FROM gold.final_reviews_test")
        existing_count = cur.fetchone()[0]
        print(f"📊 Existing records in table: {existing_count}")

        # Optional: Check for potential duplicates based on key fields
        # This prevents inserting the same review multiple times
        print("🔍 Checking for potential duplicates...")
        
        # Create a temporary table with new data for duplicate checking
        cur.execute("DROP TABLE IF EXISTS temp_new_reviews")
        cur.execute("""
        CREATE TEMP TABLE temp_new_reviews (
            city TEXT,
            hotel_name TEXT,
            reviewer_name TEXT,
            review_date TEXT,
            positive_review TEXT,
            negative_review TEXT
        )
        """)
        
        # Insert new data into temp table for comparison
        temp_insert_sql = """
        INSERT INTO temp_new_reviews (city, hotel_name, reviewer_name, review_date, positive_review, negative_review)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        
        temp_data = []
        for _, row in final_df.iterrows():
            temp_data.append((
                row['city'], row['hotel_name'], row['reviewer_name'],
                row['review_date'], row['positive_review'], row['negative_review']
            ))
        
        cur.executemany(temp_insert_sql, temp_data)
        
        # Find records that don't already exist (basic duplicate prevention)
        cur.execute("""
        SELECT COUNT(*) FROM temp_new_reviews t
        WHERE NOT EXISTS (
            SELECT 1 FROM gold.final_reviews_test f
            WHERE f.city = t.city 
            AND f.hotel_name = t.hotel_name
            AND f.reviewer_name = t.reviewer_name
            AND f.review_date = t.review_date
        )
        """)
        
        new_records_count = cur.fetchone()[0]
        duplicate_count = len(final_df) - new_records_count
        
        print(f"📝 New records to insert: {new_records_count}")
        print(f"🔄 Potential duplicates skipped: {duplicate_count}")

        # Insert only non-duplicate data
        if new_records_count > 0:
            insert_sql = """
            INSERT INTO gold.final_reviews_test (
                city, hotel_name, reviewer_name, reviewer_nationality, duration,
                check_in_date, review_date, travel_type, room_type, positive_review,
                negative_review, sentiment_classification, positive_tokens, negative_tokens
            )
            SELECT %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            WHERE NOT EXISTS (
                SELECT 1 FROM gold.final_reviews_test f
                WHERE f.city = %s 
                AND f.hotel_name = %s
                AND f.reviewer_name = %s
                AND f.review_date = %s
            )
            """

            batch_size = 1000
            inserted_count = 0
            
            for i in tqdm(range(0, len(final_df), batch_size), desc="Inserting new data"):
                batch = final_df.iloc[i:i+batch_size]
                
                for _, row in batch.iterrows():
                    # Data for insertion + duplicate check
                    insert_data = (
                        row['city'], row['hotel_name'], row['reviewer_name'], 
                        row['reviewer_nationality'], row['duration'], row['check_in_date'],
                        row['review_date'], row['travel_type'], row['room_type'],
                        row['positive_review'], row['negative_review'], 
                        row['sentiment_classification'], row['positive_tokens'], 
                        row['negative_tokens'],
                        # Duplicate check parameters
                        row['city'], row['hotel_name'], row['reviewer_name'], row['review_date']
                    )
                    
                    cur.execute(insert_sql, insert_data)
                    if cur.rowcount > 0:
                        inserted_count += 1
                
                conn.commit()

            print(f"✅ Records actually inserted: {inserted_count}")
        else:
            print("ℹ️ No new records to insert (all were duplicates)")

        # Verify final count
        cur.execute("SELECT COUNT(*) FROM gold.final_reviews_test")
        final_count = cur.fetchone()[0]
        print(f"📊 Final table count: {final_count}")
        print(f"📈 Net new records added: {final_count - existing_count}")
        
        # Close connection
        cur.close()
        conn.close()

    except Exception as e:
        print(f"❌ Database ingestion failed: {e}")
        final_df.to_csv('backup_final_reviews.csv', index=False)
        print("✅ Backup saved to CSV")
        
        # Clean up connection
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()
else:
    print("No data to ingest")

print("✅ ETL Pipeline completed!")

=== DATABASE INGESTION (APPEND MODE) ===
✅ Target table ready (created if not exists)
📊 Existing records in table: 0
🔍 Checking for potential duplicates...
✅ Target table ready (created if not exists)
📊 Existing records in table: 0
🔍 Checking for potential duplicates...
📝 New records to insert: 23
🔄 Potential duplicates skipped: 0
📝 New records to insert: 23
🔄 Potential duplicates skipped: 0


Inserting new data: 100%|██████████| 1/1 [00:02<00:00,  2.44s/it]



✅ Records actually inserted: 23
📊 Final table count: 23
📈 Net new records added: 23
✅ ETL Pipeline completed!
